# LDA Training
<figure>
<img src=https://s2.loli.net/2022/02/28/X7vzOlDHJtP6UnM.png style="width: 600px">
<figcaption>The LDA training algorithm from <a href=http://www.arbylon.net/publications/text-est.pdf>Parameter estimation for text analysis</a></figcaption>
</figure>

In [1]:
import random
import numpy as np
from collections import defaultdict, OrderedDict
from types import SimpleNamespace
from tqdm.notebook import tqdm
from visualize import visualize_topic_word

In [4]:
# === corpus loading ===
class NeurIPSCorpus:
    def __init__(self, data_path, num_topics, max_num_docs=100, max_num_words=10000, max_doc_length=1000):
        self.docs = []
        self.word2id = OrderedDict()
        self.max_doc_length = max_doc_length

        # only keep the most frequent words
        word2cnt = defaultdict(int)
        with open(data_path) as fin:
            for i, line in enumerate(fin):
                if i >= max_num_docs: break
                for word in line.strip().split():
                    word2cnt[word] += 1
        
        word2cnt = sorted(list(word2cnt.items()), key=lambda x: x[1], reverse=True)
        if len(word2cnt) > max_num_words:
            word2cnt = word2cnt[:max_num_words]
        word2cnt = dict(word2cnt)

        # read in the doc and convert words to integers
        with open(data_path) as fin:
            for i, line in enumerate(fin):
                if i >= max_num_docs: break
                doc = []
                for word in line.strip().split():
                    if len(doc) >= self.max_doc_length: break
                    if word not in word2cnt: continue
                    if word not in self.word2id: 
                        self.word2id[word] = len(self.word2id)
                    doc.append(self.word2id[word])
                self.docs.append(doc)

        self.num_docs = len(self.docs)
        self.num_topics = num_topics
        self.num_words = len(self.word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}
        print(
            "num_docs:", self.num_docs, 
            "num_topics:", self.num_topics, 
            "num_words:", self.num_words
        )

corpus = NeurIPSCorpus(
    data_path="data/papers.txt", 
    num_topics=10,
    max_num_docs=1000,
    max_num_words=10000,
    max_doc_length=200,
)
hparams = SimpleNamespace(
    alpha=np.ones([corpus.num_topics], dtype=float) / corpus.num_topics,
    beta = np.ones([corpus.num_words], dtype=float) / corpus.num_topics,
    gibbs_sampling_max_iters=500,
)

num_docs: 1000 num_topics: 10 num_words: 7882


In [5]:
# === initialization ===
print("Initializing...", flush=True)
n_doc_topic = np.zeros([corpus.num_docs, corpus.num_topics], dtype=float) # n_m^(k)
n_topic_word = np.zeros([corpus.num_topics, corpus.num_words], dtype=float) # n_k^(t)
z_doc_word = np.zeros([corpus.num_docs, corpus.max_doc_length], dtype=int)

for doc_i in range(corpus.num_docs):
    for j, word_j in enumerate(corpus.docs[doc_i]):
        topic_ij = random.randint(0, corpus.num_topics - 1)
        n_doc_topic[doc_i, topic_ij] += 1
        n_topic_word[topic_ij, word_j] += 1
        z_doc_word[doc_i, j] = topic_ij

# === Gibbs sampling ===
print("Gibbs sampling...", flush=True)
for iteration in tqdm(range(hparams.gibbs_sampling_max_iters)):
    for doc_i in range(corpus.num_docs):
        for j, word_j in enumerate(corpus.docs[doc_i]):
            # remove the old assignment
            topic_ij = z_doc_word[doc_i, j]
            n_doc_topic[doc_i, topic_ij] -= 1
            n_topic_word[topic_ij, word_j] -= 1
            # compute the new assignment
            p_doc_topic = (n_doc_topic[doc_i, :] + hparams.alpha) \
                        / np.sum(n_doc_topic[doc_i] + hparams.alpha)
            p_topic_word = (n_topic_word[:, word_j] + hparams.beta[word_j]) \
                        / np.sum(n_topic_word + hparams.beta, axis=1)
            p_topic = p_doc_topic * p_topic_word
            p_topic /= np.sum(p_topic)
            # record the new assignment
            new_topic_ij = np.random.choice(np.arange(corpus.num_topics), p=p_topic)
            n_doc_topic[doc_i, new_topic_ij] += 1
            n_topic_word[new_topic_ij, word_j] += 1
            z_doc_word[doc_i, j] = new_topic_ij

    if iteration % 50 == 0:
        print(f"Iter [{iteration}]===")
        # === Check convergence and read out parameters ===
        theta = (n_doc_topic + hparams.alpha) / np.sum(n_doc_topic + hparams.alpha, axis=1, keepdims=True)
        phi = (n_topic_word + hparams.beta) / np.sum(n_topic_word + hparams.beta, axis=1, keepdims=True)

        all_top_words = []
        all_top_probs = []
        for topic in range(corpus.num_topics):
            top_words = np.argsort(phi[topic])[::-1][:10]
            top_probs = phi[topic, top_words]
            top_words = [corpus.id2word[word] for word in top_words]
            all_top_words.append(top_words)
            all_top_probs.append(top_probs)
            print(f"Topic {topic}:", top_words)
        visualize_topic_word(all_top_words, all_top_probs)

Initializing...
Gibbs sampling...


Iter [0]===
Topic 0: ['model', 'network', 'algorithm', 'problem', 'learning', 'learn', 'function', 'neural', 'system', 'datum']
Topic 1: ['model', 'algorithm', 'network', 'neural', 'system', 'input', 'problem', 'datum', 'function', 'learning']
Topic 2: ['model', 'algorithm', 'network', 'problem', 'neural', 'system', 'datum', 'method', 'set', 'result']
Topic 3: ['network', 'model', 'learning', 'algorithm', 'time', 'system', 'problem', 'method', 'process', 'university']
Topic 4: ['model', 'learning', 'learn', 'algorithm', 'network', 'neural', 'problem', 'introduction', 'function', 'system']
Topic 5: ['model', 'network', 'problem', 'function', 'algorithm', 'learning', 'datum', 'abstract', 'base', 'system']
Topic 6: ['model', 'network', 'neural', 'algorithm', 'learning', 'abstract', 'problem', 'function', 'datum', 'university']
Topic 7: ['model', 'network', 'neural', 'problem', 'datum', 'function', 'algorithm', 'method', 'learn', 'learning']
Topic 8: ['model', 'network', 'algorithm', 'func

Iter [50]===
Topic 0: ['algorithm', 'training', 'network', 'error', 'learning', 'set', 'problem', 'learn', 'classification', 'input']
Topic 1: ['network', 't', 'neural', 'e', 'x', 'analog', 'circuit', 'neuron', 'd', 'l']
Topic 2: ['signal', 'information', 'neuron', 'spike', 'code', 'time', 'temporal', 'noise', 'edu', 'auditory']
Topic 3: ['problem', 'algorithm', 'state', 'function', 'method', 'decision', 'learning', 'value', 'edu', 'policy']
Topic 4: ['learning', 'learn', 'network', 'system', 'time', 'model', 'dynamic', 'control', 'state', 'neural']
Topic 5: ['x', 'function', 'datum', 'vector', 'kernel', 'e', 'linear', 'k', 't', 'n']
Topic 6: ['system', 'recognition', 'speech', 'network', 'neural', 'base', 'word', 'paper', 'approach', 'character']
Topic 7: ['model', 'visual', 'cell', 'cortex', 'neuron', 'pattern', 'response', 'system', 'cortical', 'eye']
Topic 8: ['image', 'object', 'visual', 'representation', 'model', 'face', 'feature', 'human', 'recognition', 'vision']
Topic 9: ['mod

Iter [100]===
Topic 0: ['algorithm', 'training', 'problem', 'network', 'learning', 'set', 'method', 'error', 'classification', 'learn']
Topic 1: ['network', 't', 'e', 'neural', 'd', 'function', 'analog', 'x', 'unit', 'n']
Topic 2: ['signal', 'information', 'neuron', 'spike', 'time', 'code', 'model', 'noise', 'temporal', 'auditory']
Topic 3: ['problem', 'algorithm', 'state', 'learning', 'reinforcement', 'function', 'decision', 'value', 'time', 'policy']
Topic 4: ['learning', 'network', 'learn', 'system', 'model', 'neural', 'dynamic', 'time', 'rule', 'control']
Topic 5: ['vector', 'x', 'function', 'datum', 'n', 'algorithm', 'linear', 'kernel', 'analysis', 'component']
Topic 6: ['system', 'recognition', 'network', 'speech', 'neural', 'time', 'computer', 'application', 'character', 'word']
Topic 7: ['model', 'visual', 'cell', 'neuron', 'cortex', 'system', 'field', 'input', 'activity', 'synaptic']
Topic 8: ['image', 'object', 'representation', 'visual', 'model', 'feature', 'human', 'face', 

Iter [150]===
Topic 0: ['algorithm', 'training', 'problem', 'method', 'error', 'function', 'set', 'learning', 'classification', 'learn']
Topic 1: ['network', 't', 'neural', 'e', 'd', 'x', 'analog', 'l', 'function', 'circuit']
Topic 2: ['signal', 'information', 'spike', 'noise', 'time', 'neuron', 'system', 'code', 'source', 'auditory']
Topic 3: ['problem', 'learning', 'state', 'algorithm', 'reinforcement', 'function', 'control', 'decision', 'value', 'time']
Topic 4: ['network', 'learning', 'model', 'learn', 'neural', 'system', 'dynamic', 'time', 'input', 'rule']
Topic 5: ['vector', 'x', 'function', 'datum', 'algorithm', 'n', 'linear', 'feature', 'space', 'kernel']
Topic 6: ['system', 'recognition', 'speech', 'network', 'neural', 'word', 'application', 'computer', 'base', 'character']
Topic 7: ['model', 'neuron', 'visual', 'cell', 'cortex', 'activity', 'input', 'university', 'field', 'cortical']
Topic 8: ['image', 'object', 'visual', 'human', 'representation', 'model', 'recognition', 'fa

...

Iter [250]===
Topic 0: ['algorithm', 'training', 'problem', 'learning', 'error', 'set', 'function', 'method', 'learn', 'classification']
Topic 1: ['t', 'x', 'e', 'neural', 'network', 'l', 'd', 'circuit', 'n', 'j']
Topic 2: ['signal', 'information', 'time', 'spike', 'noise', 'source', 'code', 'system', 'auditory', 'response']
Topic 3: ['learning', 'problem', 'state', 'algorithm', 'reinforcement', 'control', 'function', 'time', 'decision', 'value']
Topic 4: ['network', 'learning', 'neural', 'learn', 'unit', 'input', 'output', 'model', 'weight', 'algorithm']
Topic 5: ['datum', 'vector', 'x', 'feature', 'function', 'algorithm', 'space', 'linear', 'n', 'kernel']
Topic 6: ['system', 'recognition', 'speech', 'network', 'neural', 'time', 'computer', 'word', 'base', 'character']
Topic 7: ['model', 'neuron', 'cell', 'visual', 'system', 'cortex', 'synaptic', 'input', 'activity', 'cortical']
Topic 8: ['image', 'object', 'visual', 'human', 'model', 'representation', 'motion', 'task', 'face', 'recog

Iter [300]===
Topic 0: ['algorithm', 'training', 'problem', 'learning', 'function', 'method', 'error', 'set', 'learn', 'example']
Topic 1: ['t', 'x', 'e', 'd', 'l', 'j', 'n', 'analog', 'circuit', 'w']
Topic 2: ['signal', 'information', 'noise', 'source', 'spike', 'time', 'code', 'auditory', 'system', 'institute']
Topic 3: ['learning', 'problem', 'state', 'algorithm', 'control', 'reinforcement', 'function', 'learn', 'time', 'value']
Topic 4: ['network', 'neural', 'learning', 'learn', 'unit', 'weight', 'input', 'algorithm', 'model', 'output']
Topic 5: ['datum', 'vector', 'x', 'function', 'algorithm', 'feature', 'analysis', 'kernel', 'space', 'basis']
Topic 6: ['system', 'recognition', 'speech', 'network', 'neural', 'time', 'application', 'computer', 'word', 'character']
Topic 7: ['neuron', 'model', 'cell', 'visual', 'cortex', 'system', 'synaptic', 'input', 'neural', 'stimulus']
Topic 8: ['image', 'object', 'visual', 'model', 'human', 'representation', 'motion', 'face', 'task', 'feature']

Iter [350]===
Topic 0: ['algorithm', 'training', 'learning', 'problem', 'error', 'function', 'method', 'learn', 'classification', 'example']
Topic 1: ['x', 't', 'e', 'j', 'l', 'd', 'f', 'circuit', 'analog', 'w']
Topic 2: ['signal', 'information', 'time', 'source', 'noise', 'system', 'code', 'spike', 'auditory', 'institute']
Topic 3: ['learning', 'problem', 'state', 'control', 'algorithm', 'reinforcement', 'function', 'learn', 'decision', 'value']
Topic 4: ['network', 'neural', 'learning', 'weight', 'input', 'unit', 'learn', 'output', 'algorithm', 'function']
Topic 5: ['vector', 'datum', 'algorithm', 'x', 'feature', 'function', 'space', 'kernel', 'point', 'linear']
Topic 6: ['system', 'recognition', 'network', 'speech', 'neural', 'time', 'computer', 'base', 'word', 'character']
Topic 7: ['model', 'neuron', 'cell', 'visual', 'cortex', 'input', 'synaptic', 'activity', 'cortical', 'stimulus']
Topic 8: ['image', 'object', 'visual', 'model', 'human', 'motion', 'task', 'representation', 'face

Iter [400]===
Topic 0: ['algorithm', 'training', 'learning', 'problem', 'error', 'function', 'method', 'learn', 'classification', 'set']
Topic 1: ['t', 'x', 'e', 'w', 'l', 'd', 'f', 'circuit', 'n', 'j']
Topic 2: ['signal', 'information', 'source', 'noise', 'time', 'code', 'auditory', 'channel', 'edu', 'frequency']
Topic 3: ['learning', 'state', 'problem', 'control', 'algorithm', 'reinforcement', 'learn', 'function', 'time', 'value']
Topic 4: ['network', 'neural', 'learning', 'input', 'weight', 'unit', 'learn', 'output', 'algorithm', 'number']
Topic 5: ['datum', 'vector', 'algorithm', 'function', 'feature', 'x', 'kernel', 'space', 'problem', 'method']
Topic 6: ['system', 'network', 'recognition', 'neural', 'speech', 'time', 'computer', 'word', 'base', 'character']
Topic 7: ['neuron', 'model', 'cell', 'system', 'cortex', 'activity', 'synaptic', 'neural', 'visual', 'stimulus']
Topic 8: ['image', 'object', 'visual', 'model', 'human', 'motion', 'representation', 'face', 'task', 'edu']
Topic

Iter [450]===
Topic 0: ['algorithm', 'training', 'learning', 'problem', 'error', 'method', 'classification', 'learn', 'example', 'set']
Topic 1: ['t', 'x', 'e', 'd', 'n', 'l', 'f', 'circuit', 'analog', 'w']
Topic 2: ['signal', 'information', 'noise', 'source', 'auditory', 'code', 'time', 'channel', 'system', 'edu']
Topic 3: ['learning', 'problem', 'state', 'control', 'algorithm', 'reinforcement', 'learn', 'function', 'decision', 'value']
Topic 4: ['network', 'neural', 'learning', 'input', 'unit', 'learn', 'algorithm', 'function', 'output', 'weight']
Topic 5: ['datum', 'vector', 'algorithm', 'function', 'feature', 'space', 'x', 'point', 'method', 'kernel']
Topic 6: ['system', 'recognition', 'network', 'neural', 'speech', 'time', 'computer', 'word', 'base', 'pattern']
Topic 7: ['neuron', 'model', 'cell', 'synaptic', 'cortex', 'input', 'system', 'activity', 'stimulus', 'visual']
Topic 8: ['image', 'object', 'visual', 'model', 'human', 'motion', 'task', 'representation', 'face', 'feature']